# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
import json
import os
import sys
import glob
from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
import numpy
import pickle
import matplotlib.pyplot as plt

import jax
from jax import jit
from noisyopt import minimizeSPSA, minimizeCompass

from discopy.quantum import Circuit
from discopy.tensor import Tensor
from discopy.utils import loads
#from pytket.extensions.qiskit import AerBackend
#from pytket.extensions.qulacs import QulacsBackend
#from pytket.extensions.cirq import CirqStateSampleBackend
backend = None

from utils import get_symbols, construct_data_and_labels, select_circuits, read_diagrams, create_labeled_classes, bin_class_loss, multi_class_loss, bin_class_acc, multi_class_acc, visualize_result_noisyopt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings('ignore')
this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

EPOCHS = 500
SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

rng = numpy.random.default_rng(SEED)
numpy.random.seed(SEED)

## Read circuit data

We read the circuits from the pickled files. Select if we perform binary classification or multi-class classification. Give number of qubits to create classes:
- 1 qubits -> 2^1 = 2 classes i.e. binary classification
- 2 qubits -> 2^2 = 4 classes
- ...
- 5 qubits -> 2^5 = 32 classes, etc.

In [2]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 1
layers = 1
single_qubit_params = 3
n_wire_count = 1

loss = multi_class_loss
acc = multi_class_acc

if classification == 1:
    loss = bin_class_loss
    acc = bin_class_acc

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [3]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [4]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Lambeq optimizer

## Model

In [5]:
def make_pred_fn(circuits):
    # In the case we want to use other backends. 
    # Currently does not work properly.
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

## Loss function and evaluation

In [6]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)

        cost = loss(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Minimization with noisyopt

In [7]:
def initialize_parameters(old_params, old_values, new_params):
    new_values = list(numpy.array(rng.random(len(new_params))))
    old_param_dict = {}
    for p, v in zip(old_params, old_values):
        old_param_dict[p] = v
        
    parameters = sorted(set(old_params + new_params), key=default_sort_key)
    values = []
    for p in parameters:
        if p in old_param_dict:
            values.append(old_param_dict[p])
        else:
            values.append(new_values.pop())
            
    return parameters, np.array(values)

In [8]:
EPOCHS = 4000
initial_number_of_circuits = 44
syms = {}
limit = False
all_training_keys = list(training_circuits.keys())
initial_circuit_keys = all_training_keys[:initial_number_of_circuits]
current_training_circuits = {}
result_file = workload_size + "_noisyopt_2" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

for k in initial_circuit_keys:
    current_training_circuits[k] = training_circuits[k]
    
syms = get_symbols(current_training_circuits)
parameters = sorted(syms, key=default_sort_key)
if initial_number_of_circuits > 5 and os.path.exists("points//" + result_file + ".npz"):
    with open("points//" + result_file + ".npz", "rb") as f:
        print("Loading parameters from file " + result_file)
        npzfile = np.load(f)
        init_params_spsa = npzfile['arr_0']
else:
    print("Initializing new parameters")
    init_params_spsa = np.array(rng.random(len(parameters)))
result = None
run = 0

Loading parameters from file main_noisyopt_21_1_3


In [ ]:
for i, key in enumerate(all_training_keys[initial_number_of_circuits:]):
    print("Progress: ", round((i + initial_number_of_circuits)/len(all_training_keys), 3))
    
    if len(syms) == len(get_symbols(current_training_circuits)) and i > 0:
        if i != len(all_training_keys[1:]):
            current_training_circuits[key] = training_circuits[key]
            new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
            if result:
                parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)
                #continue
            else:
                syms = get_symbols(current_training_circuits)
                parameters = sorted(syms, key=default_sort_key)
                init_params_spsa = np.array(rng.random(len(parameters)))
    else:
        run += 1
    
    # Select those circuits from test and validation circuits which share the parameters with the current training circuits
    current_validation_circuits = select_circuits(current_training_circuits, validation_circuits)
    current_test_circuits = select_circuits(current_training_circuits, test_circuits)
    
    if len(current_validation_circuits) == 0 or len(current_test_circuits) == 0:
        continue
    
    # Create lists with circuits and their corresponding label
    training_circuits_l, training_data_labels_l = construct_data_and_labels(current_training_circuits, training_data_labels)
    validation_circuits_l, validation_data_labels_l = construct_data_and_labels(current_validation_circuits, validation_data_labels)
    test_circuits_l, test_data_labels_l = construct_data_and_labels(current_test_circuits, test_data_labels)
    
    # Limit the number of validation and test circuits to 20% of number of the training circuits
    if limit:
        val_test_circ_size = ceil(len(current_training_circuits))
        if len(current_validation_circuits) > val_test_circ_size:
            validation_circuits_l = validation_circuits_l[:val_test_circ_size]
            validation_data_labels_l = validation_data_labels_l[:val_test_circ_size]
        if len(current_test_circuits) > val_test_circ_size:
            test_circuits_l = test_circuits_l[:val_test_circ_size]
            test_data_labels_l = test_data_labels_l[:val_test_circ_size]
    
    stats = f"Number of training circuits: {len(training_circuits_l)}   "\
        + f"Number of validation circuits: {len(validation_circuits_l)}   "\
        + f"Number of test circuits: {len(test_circuits_l)}   "\
        + f"Number of parameters in model: {len(set([sym for circuit in training_circuits_l for sym in circuit.free_symbols]))}"
    
    with open("results//" + result_file + ".txt", "a") as f:
        f.write(stats + "\n")
    
    print(stats)
    
    train_pred_fn = jit(make_pred_fn(training_circuits_l))
    dev_pred_fn = jit(make_pred_fn(validation_circuits_l))
    test_pred_fn = make_pred_fn(test_circuits_l)
    
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)
    
    def callback_fn(xk):
        #print(xk)
        valid_loss = dev_cost_fn(xk)
        train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
        train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
        valid_acc = numpy.around(float(dev_accs[-1]), 4)
        iters = int(len(train_accs)/2)
        if iters % 200 == 0:
            info = f"Epoch: {iters}   "\
            + f"train/loss: {train_loss}   "\
            + f"valid/loss: {numpy.around(float(valid_loss), 4)}   "\
            + f"train/acc: {train_acc}   "\
            + f"valid/acc: {valid_acc}"
        
            with open("results//" + result_file + ".txt", "a") as f:
                f.write(info + "\n")
                
            print(info, file=sys.stderr)
        return valid_loss
    
    a_value = 0.0053
    c_value = 0.0185
            
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)

    result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a = a_value, c = c_value, niter=EPOCHS, callback=callback_fn)
    #result = minimizeCompass(train_cost_fn, x0=init_params_spsa, redfactor=2.0, deltainit=1.0, deltatol=0.001, feps=1e-15, errorcontrol=True, funcNinit=30, funcmultfactor=2.0, paired=True, alpha=0.05, callback=callback_fn)

    figure_path = this_folder + "//results//" + result_file + ".png"
    visualize_result_noisyopt(result, make_cost_fn, test_pred_fn, test_data_labels_l, train_costs, train_accs, dev_costs, dev_accs, figure_path, result_file)
    
    run += 1
    #EPOCHS += 100
    syms = get_symbols(current_training_circuits)
    
    # Extend for the next optimization round
    current_training_circuits[key] = training_circuits[key]
    new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
    parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)

Progress:  0.098
Number of training circuits: 43   Number of validation circuits: 104   Number of test circuits: 104   Number of parameters in model: 235


Epoch: 200   train/loss: 0.1977   valid/loss: 0.7977   train/acc: 0.9767   valid/acc: 0.6058
Epoch: 400   train/loss: 0.1716   valid/loss: 0.7816   train/acc: 0.9535   valid/acc: 0.6058
Epoch: 600   train/loss: 0.1723   valid/loss: 0.7858   train/acc: 0.9302   valid/acc: 0.5962
Epoch: 800   train/loss: 0.162   valid/loss: 0.786   train/acc: 0.9535   valid/acc: 0.6058
Epoch: 1000   train/loss: 0.1625   valid/loss: 0.7766   train/acc: 0.9767   valid/acc: 0.6058
Epoch: 1200   train/loss: 0.1778   valid/loss: 0.7748   train/acc: 0.9535   valid/acc: 0.5962
Epoch: 1400   train/loss: 0.1419   valid/loss: 0.7781   train/acc: 0.9535   valid/acc: 0.5962
Epoch: 1600   train/loss: 0.1569   valid/loss: 0.7742   train/acc: 0.9535   valid/acc: 0.5962
Epoch: 1800   train/loss: 0.1583   valid/loss: 0.7794   train/acc: 0.9535   valid/acc: 0.5962
Epoch: 2000   train/loss: 0.1571   valid/loss: 0.7787   train/acc: 0.9535   valid/acc: 0.5962
Epoch: 2200   train/loss: 0.165   valid/loss: 0.7762   train/acc: 

Test accuracy: 0.6346154
Progress:  0.1
Number of training circuits: 45   Number of validation circuits: 104   Number of test circuits: 104   Number of parameters in model: 235


Epoch: 200   train/loss: 0.1611   valid/loss: 0.7611   train/acc: 0.9556   valid/acc: 0.5865
Epoch: 400   train/loss: 0.1598   valid/loss: 0.7688   train/acc: 0.9556   valid/acc: 0.5865
Epoch: 600   train/loss: 0.1825   valid/loss: 0.7681   train/acc: 0.9333   valid/acc: 0.5865
Epoch: 800   train/loss: 0.1548   valid/loss: 0.7689   train/acc: 0.9556   valid/acc: 0.5865
Epoch: 1000   train/loss: 0.1455   valid/loss: 0.7701   train/acc: 0.9333   valid/acc: 0.5865
Epoch: 1200   train/loss: 0.1652   valid/loss: 0.7719   train/acc: 0.9556   valid/acc: 0.5865
Epoch: 1400   train/loss: 0.1346   valid/loss: 0.7717   train/acc: 0.9556   valid/acc: 0.5865
Epoch: 1600   train/loss: 0.1767   valid/loss: 0.7703   train/acc: 0.9556   valid/acc: 0.5865
Epoch: 1800   train/loss: 0.1428   valid/loss: 0.7705   train/acc: 0.9778   valid/acc: 0.5865
Epoch: 2000   train/loss: 0.1542   valid/loss: 0.7734   train/acc: 0.9556   valid/acc: 0.5865
Epoch: 2200   train/loss: 0.183   valid/loss: 0.7708   train/acc

Test accuracy: 0.6346154
Progress:  0.103
Number of training circuits: 46   Number of validation circuits: 104   Number of test circuits: 104   Number of parameters in model: 235


...

Epoch: 200   train/loss: 0.1855   valid/loss: 0.7675   train/acc: 0.9545   valid/acc: 0.6667
Epoch: 400   train/loss: 0.16   valid/loss: 0.7681   train/acc: 0.9545   valid/acc: 0.6667
Epoch: 600   train/loss: 0.1432   valid/loss: 0.7583   train/acc: 0.9394   valid/acc: 0.6667
Epoch: 800   train/loss: 0.1443   valid/loss: 0.7743   train/acc: 0.9394   valid/acc: 0.6574
Epoch: 1000   train/loss: 0.1326   valid/loss: 0.7766   train/acc: 0.9697   valid/acc: 0.6574
Epoch: 1200   train/loss: 0.1403   valid/loss: 0.7889   train/acc: 0.9848   valid/acc: 0.6574
Epoch: 1400   train/loss: 0.1361   valid/loss: 0.7636   train/acc: 0.9545   valid/acc: 0.6574
Epoch: 1600   train/loss: 0.1732   valid/loss: 0.7639   train/acc: 0.9545   valid/acc: 0.6574
Epoch: 1800   train/loss: 0.1382   valid/loss: 0.7832   train/acc: 0.9545   valid/acc: 0.6574
Epoch: 2000   train/loss: 0.1736   valid/loss: 0.7674   train/acc: 0.9545   valid/acc: 0.6574
Epoch: 2200   train/loss: 0.1378   valid/loss: 0.7624   train/acc:

Test accuracy: 0.6574074
Progress:  0.15
Number of training circuits: 67   Number of validation circuits: 108   Number of test circuits: 108   Number of parameters in model: 238


Epoch: 200   train/loss: 0.1605   valid/loss: 0.7642   train/acc: 0.9552   valid/acc: 0.6667
Epoch: 400   train/loss: 0.1596   valid/loss: 0.7667   train/acc: 0.9552   valid/acc: 0.6667
Epoch: 600   train/loss: 0.1741   valid/loss: 0.7533   train/acc: 0.9701   valid/acc: 0.6667
Epoch: 800   train/loss: 0.1731   valid/loss: 0.7708   train/acc: 0.9701   valid/acc: 0.6667
Epoch: 1000   train/loss: 0.1382   valid/loss: 0.7679   train/acc: 0.9701   valid/acc: 0.6667
Epoch: 1200   train/loss: 0.176   valid/loss: 0.7573   train/acc: 0.9552   valid/acc: 0.6667
Epoch: 1400   train/loss: 0.18   valid/loss: 0.756   train/acc: 0.9552   valid/acc: 0.6667
Epoch: 1600   train/loss: 0.1401   valid/loss: 0.7572   train/acc: 0.9701   valid/acc: 0.6667
Epoch: 1800   train/loss: 0.1509   valid/loss: 0.7566   train/acc: 0.9851   valid/acc: 0.6667
Epoch: 2000   train/loss: 0.1279   valid/loss: 0.7634   train/acc: 0.9552   valid/acc: 0.6667
Epoch: 2200   train/loss: 0.1277   valid/loss: 0.7597   train/acc: 0

Test accuracy: 0.6574074
Progress:  0.152
Number of training circuits: 68   Number of validation circuits: 108   Number of test circuits: 108   Number of parameters in model: 238


Epoch: 200   train/loss: 0.1975   valid/loss: 0.7608   train/acc: 0.9559   valid/acc: 0.6667
Epoch: 400   train/loss: 0.14   valid/loss: 0.7571   train/acc: 0.9412   valid/acc: 0.6667
Epoch: 600   train/loss: 0.1712   valid/loss: 0.759   train/acc: 0.9559   valid/acc: 0.6667
Epoch: 800   train/loss: 0.1411   valid/loss: 0.7683   train/acc: 0.9559   valid/acc: 0.6667
Epoch: 1000   train/loss: 0.1867   valid/loss: 0.7708   train/acc: 0.9559   valid/acc: 0.6574
Epoch: 1200   train/loss: 0.1445   valid/loss: 0.7541   train/acc: 0.9412   valid/acc: 0.6667
Epoch: 1400   train/loss: 0.1422   valid/loss: 0.7615   train/acc: 0.9706   valid/acc: 0.6574
Epoch: 1600   train/loss: 0.1326   valid/loss: 0.7608   train/acc: 0.9559   valid/acc: 0.6481
Epoch: 1800   train/loss: 0.155   valid/loss: 0.7625   train/acc: 0.9559   valid/acc: 0.6574
Epoch: 2000   train/loss: 0.1425   valid/loss: 0.7694   train/acc: 0.9559   valid/acc: 0.6667
Epoch: 2200   train/loss: 0.1342   valid/loss: 0.7673   train/acc: 0